### Imports

In [1]:
# Run the below commands if you don't already have Pandas or TextBlob
#!pip install np
#!pip install textblob

import np
import re

# for handling data
import pandas as pd

# for graphs
import networkx as nx

#for sentiment analysis
from textblob import TextBlob

Now get the scraped Twitter data from the .csv files

In [7]:
files = ['Polkamon_March_April_2021_data', 'Polkamon_May_June_2021_data']

In [8]:
# calculate the sentiment of each tweet using TextBlob
def sentiment_analysis(tweets):
    tweets["sentiment_score"] = tweets["Text"].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
    tweets["sentiment"] = np.select([tweets["sentiment_score"] < 0,
                                     tweets["sentiment_score"] == 0,
                                     tweets["sentiment_score"] > 0],
                                    ['neg', 'neu', 'pos'])
    aggregated = tweets[["Username", "sentiment_score"]].groupby("Username").agg(sentiment_sum=('sentiment_score', 'sum'),
                                                                                 tweet_count=('Username', 'count'))
    return tweets, aggregated

In [9]:
# extract user mentions from tweet text
def get_mentions(tweets):
    tweets["user_mentions"] = tweets["Text"].apply(lambda x: re.findall(r"@(\w+)", x))
    return tweets

In [11]:
for file in files:
    df = pd.read_csv(f"{file}.csv")
    df, aggdf = sentiment_analysis(df)
    df = get_mentions(df)

    df.to_csv(f"{file}_sentiment.csv")
    aggdf.to_csv(f"{file}_per_user_sentiment.csv")
    aggdf.head(10)